In [1]:
import torch

In [7]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text=f.read()
#print(text[:200])
chars=sorted(set(text))
print(chars)
#tokenizer consist of encoder and decoder 
#encoder-convert each element in this array to an integer eg- maybe '\n' would be 0
print(len(chars))


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [6]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encoded=encode('hello')
#decoded=decode(encoded)
#print(decoded)
#so these are character level tokenizers which takes each character and coverts it to integer equivalent.

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80, 44, 61, 58,  1, 40, 71, 68, 63, 58, 56, 73,  1, 31, 74, 73, 58, 67,
        55, 58, 71, 60,  1, 58, 26, 68, 68, 64,  1, 68, 59,  1, 28, 68, 71, 68,
        73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,
         1, 62, 67,  1, 39, 79,  0,  1,  1,  1,  1,  0, 44, 61, 62, 72,  1, 58,
        55, 68, 68, 64,  1, 62, 72,  1, 59, 68, 71,  1, 73, 61, 58,  1, 74, 72,
        58,  1, 68, 59,  1, 54, 67, 78, 68, 67])
